In [1]:
from reading_data import reading_files, reading_files_test
from create_dataset import create_dataset
from UMLS_methods import *
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp "/content/drive/My Drive/meta.bin" meta.bin
!cp "/content/drive/My Drive/model.bin" model.bin

In [ ]:
import torch, joblib
m = torch.load("model.bin")
meta = joblib.load("meta.bin")
enc_label = meta['enc_label']

In [2]:
# load training data
train_data, CUI, iCUI = reading_files("C:/Users/monil/Desktop/BMI 598 - NLP/Project/Clinical-Entity-Normalization/train")
train_df = create_dataset(train_data)
### removing CUIs from dataset that have only one mentions
train_single_cui = []
for cui, mention in CUI.items():
    if len(mention) == 1:
        train_single_cui.append(cui)

Total number of files:  50
Total CUIs:  2331
Total Data:  50


In [54]:
# load test data
test_data, _ = reading_files_test("C:/Users/monil/Desktop/BMI 598 - NLP/Project/Clinical-Entity-Normalization/testing")
test_df = create_dataset(test_data)

Total number of files:  50


In [55]:
test_df.head()

,cui,original_sentence,masked_sentence,mention,position_start,position_end
0,C0019699,004668411 CTMC 68299235 763052 9/29/1993 12:00...,004668411 CTMC 68299235 763052 9/29/1993 12:00...,HIV positive,232,244
1,C0238552,004668411 CTMC 68299235 763052 9/29/1993 12:00...,004668411 CTMC 68299235 763052 9/29/1993 12:00...,left upper quadrant pain,280,304
2,C0027497,004668411 CTMC 68299235 763052 9/29/1993 12:00...,004668411 CTMC 68299235 763052 9/29/1993 12:00...,nausea,316,322
3,C0042963,004668411 CTMC 68299235 763052 9/29/1993 12:00...,004668411 CTMC 68299235 763052 9/29/1993 12:00...,vomiting,327,335
4,CUI-less,004668411 CTMC 68299235 763052 9/29/1993 12:00...,004668411 CTMC 68299235 763052 9/29/1993 12:00...,a long-standing complaint,345,370


In [72]:
test_df['prediction'] = None
test_df['prediction_source'] = None
I = 7000


# if mention in CUI less, give CUI less
# so iterating all test CUIs and assign the CUI-less

tgt = get_tgt()
st = get_st(tgt)

for i in tqdm(test_df.index[I:], total=len(test_df)):
    I = i
    tgt = get_tgt()
    st = get_st(tgt)
    if i%50 == 0:
#         tgt = get_tgt()
        print(test_df.groupby(by='prediction_source', as_index=False).count()[['prediction_source', 'cui']])
    mention = test_df.loc[i]['mention']
    if mention in iCUI:   # iterating train data
        if 'CUI-less' in iCUI[mention]:
            test_df.at[i, 'prediction'] = 'CUI-less'
            test_df.at[i, 'prediction_source'] = 'train_data'
        elif len(iCUI[mention])==1:
            test_df.at[i, 'prediction'] = list(iCUI[mention])[0]
            test_df.at[i, 'prediction_source'] = 'train_data'            
        else:
            test_df.at[i, 'prediction'] = iCUI[mention]
            test_df.at[i, 'prediction_source'] = 'BERT'
    else:  # not in train_data so we will search UMLS
        CUIs = find_mention_in_UMLS_perfect_name(mention, st)
        print(mention, ":", test_df.loc[i]['cui'] ,"Exact", len(CUIs))
        if len(CUIs)>=1 and CUIs[0]['cui']!='NONE':
            if len(CUIs)==1:
                test_df.at[i, 'prediction'] = CUIs[0]['cui']
                test_df.at[i, 'prediction_source'] = 'UMLS_exact'
            else:
                test_df.at[i, 'prediction'] = [CUIs[_]['cui'] for _ in range(len(CUIs))]
                test_df.at[i, 'prediction_source'] = 'BERT'
        else:
            tgt = get_tgt()
            st = get_st(tgt)
            CUIs = find_mention_in_UMLS_partial_name(mention, st)
            print(mention, ":", test_df.loc[i]['cui'] ,"partial", len(CUIs))
            if len(CUIs)>=1 and CUIs[0]['cui']!='NONE':
                if len(CUIs)==1:
                    test_df.at[i, 'prediction'] = CUIs[0]['cui']
                    test_df.at[i, 'prediction_source'] = 'UMLS_partial'
                else:
                    test_df.at[i, 'prediction'] = [CUIs[_]['cui'] for _ in range(len(CUIs))]
                    test_df.at[i, 'prediction_source'] = 'BERT'
            else:
                test_df.at[i, 'prediction_source'] = 'Data available nowhere'
                test_df.at[i, 'prediction'] = "CUI-less"

  0%|                                                                                         | 0/7086 [00:00<?, ?it/s]

Empty DataFrame
Columns: [prediction_source, cui]
Index: []
his metabolic acidosis  : C0220981 Exact 1


  0%|                                                                               | 1/7086 [00:03<7:48:59,  3.97s/it]

his metabolic acidosis  : C0220981 partial 1
his renal function  : C2711919 Exact 1


  0%|                                                                               | 2/7086 [00:10<9:36:18,  4.88s/it]

...

Niferex 150 milligrams  : C0212018 partial 1
SIGNIFICANT  : C0750502 Exact 1


  1%|▉                                                                             | 85/7086 [06:44<7:50:10,  4.03s/it]

SIGNIFICANT  : C0750502 partial 25
PROBLEMS  : C0033213 Exact 1


  1%|▉                                                                             | 86/7086 [06:51<9:18:21,  4.79s/it]

PROBLEMS  : C0033213 partial 25


In [75]:
test_df[7000:].to_csv("temp_data.csv")

In [46]:
test_df['mention']

0                     HIV positive 
1         left upper quadrant pain 
2                           nausea 
3                         vomiting 
4        a long-standing complaint 
                   ...             
7081                   prostate CA 
7082    HISTORY OF PRESENT ILLNESS 
7083        Niferex 150 milligrams 
7084                   SIGNIFICANT 
7085                      PROBLEMS 
Name: mention, Length: 7086, dtype: object

In [65]:
test_df.to_csv("test_data.csv")

In [45]:
print(set(test_df['mention']).intersection(set(iCUI.keys())))
test_df.groupby(by='prediction_source').count()

{'admission', 'murmur', 'INTERACTION', 'on admission', 'CP', 'bruit'}


,cui,original_sentence,masked_sentence,mention,position_start,position_end,prediction
prediction_source,,,,,,,
BERT,1,1,1,1,1,1,1
Data available nowhere,7080,7080,7080,7080,7080,7080,7080
train_data,5,5,5,5,5,5,5


In [ ]:
# we only ask bert to rank the mention that have multiple mentinos

In [ ]:
# test_candidates will be encoded representation
# test_labels will lso be encoded

test_data = TensorDataset(test_inputs, test_masks, test_labels, test_position, test_candidates)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=config.BATCH_SIZE)

In [ ]:
model.eval()
predictions , true_labels = [], []

prediction_dataloader = test_data
for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels, b_pos, b_candidates = batch
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    
    logits = logits.detach().cpu().numpy()
    # argmax, but we need someting else
    logits = np.argsort(logits, anis=1)
    preds = []
    for i in len(logits):
        for _ in logits[i]:
            if _ in b_candidates[i]:
                pred = _
                break
            else:
                pred = logits[i][0]
        preds.append(pred)
        
    label_ids = b_labels.to('cpu').numpy()
    predictions.append(preds)
    true_labels.append(label_ids)
print('DONE.')

In [ ]:
from sklearn.metrics import accuracy_score
flat_predictions = [item for sublist in predictions for item in sublist]

flat_predictions = np.argmax(flat_predictions, axis=1).flatten()


flat_true_labels = [item for sublist in true_labels for item in sublist]

print("Accuracy: ", accuracy_score(flat_true_labels, flat_predictions))